### ERIC API

In [2]:
from scripts.eric_api import EricApi
import pandas as pd

In [3]:
eric_api = EricApi()

# Define search query
search = 'subject:"Problem Solving"'

# Define fields to retrieve
allEricFields = ['id', 'title', 'author', 'source', 'publicationdateyear', 'description',
'subject', 'peerreviewed', 'abstractor', 'audience', 'authorxlink',
'e_datemodified', 'e_fulltextauth', 'e yearadded', 'educationlevel',
'identifiersgeo', 'identifierslaw', 'identifierstest', 'iescited',
'iesfunded', 'iesgrantcontractnum', 'iesgrantcontractnumxlink',
'ieslinkpublication', 'ieslinkwwcreviewguide', 'ieswwcreviewed',
'institution', 'isbn', 'issn', 'language', 'publicationtype',
'publisher', 'sourceid', 'sponsor', 'url']

# Retrieve and save all records
records = eric_api.getAllEricRecords(search, fields=allEricFields)

Search subject:"Problem Solving" returned 46,000 records
took 388.9 seconds


In [76]:
records["title"][3]

'The School Dropout Question in Cuba'

In [82]:
import pybliometrics
pybliometrics.scopus.init()

from pybliometrics.scopus import ScopusSearch

# Replace 'YOUR_API_KEY' with your actual Scopus API key
s = ScopusSearch('NEW_VALID_API_KEY')

# Search for papers by title
paper_titles = ['Accountability--The Next Deadly Nostrum in Education?', 'Training of Self-Regulatory and Problem-Solving Competence', 'The School Dropout Question in Cuba']

for title in paper_titles:
    query = s.search(rf'TITLE("{title}")')
    
    # Retrieve the first search result (assuming it's the correct paper)
    document = query.get_results().entries[0]
    
    # Print citation data
    print(f'Title: {document.title}')
    print(f'Authors: {", ".join(author.indexed_name for author in document.authors)}')
    print(f'Publication Year: {document.coverDate[:4]}')
    print(f'Citation Count: {document.citedby_count}')
    print('---')

Scopus401Error: The requestor is not authorized to access the requested view or fields of the resource

In [15]:
import requests

# URL of the PDF file
url = "https://files.eric.ed.gov/fulltext/ED599096.pdf"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open a file in binary write mode and write the PDF content to it
    with open("downloaded_file.pdf", "wb") as f:
        f.write(response.content)
    print("PDF file downloaded successfully.")
else:
    print(f"Failed to download PDF file. Status code: {response.status_code}")

PDF file downloaded successfully.


In [5]:
# records.to_csv("database/eric_records.csv", index=False)

In [4]:
records = pd.read_csv("database/eric_records.csv")
records.dropna(inplace=True)

In [13]:
def getCountsYearly(df):
    counts = {}
    min_year = df["publicationdateyear"].min()  # Get the earliest year in the DataFrame
    max_year = df["publicationdateyear"].max()  # Get the latest year in the DataFrame
    
    for _, row in df.iterrows():
        year = int(row["publicationdateyear"])
        subject_list = row["subject"].split(",")
        for subject in subject_list:
            if subject not in counts:
                counts[subject] = {}
            for y in range(year + 1, int(max_year)):  # Include all years from the earliest to the current year
                if y not in counts[subject]:
                    counts[subject][y] = 0
                counts[subject][y] += 1
    
    # Sort the years in increasing order
    for subject in counts:
        counts[subject] = {year: counts[subject][year] for year in sorted(counts[subject])}
    
    return counts


In [14]:
counts = getCountsYearly(records)

In [15]:
import json

with open("database/eric_counts.json", "w") as f:
    json.dump(counts, f)